In [2]:
# !pip install weaviate-client
# !pip install llama_index
# !pip install openai
# !pip install --upgrade pydantic==1.10.0 typing-extensions==4.5.0
# !pip install fastcore

In [34]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index.schema import TextNode, NodeRelationship, RelatedNodeInfo
from llama_index import VectorStoreIndex, StorageContext
from llama_index import get_response_synthesizer
from llama_index.indices.postprocessor import SimilarityPostprocessor, PrevNextNodePostprocessor
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
import weaviate, os
from weaviate import EmbeddedOptions
from openai import OpenAI
import pandas as pd
import numpy as np
from llama_index import ServiceContext
from llama_index.llms import OpenAI as OpenAILLama
from llama_index.embeddings import OpenAIEmbedding
from configparser import ConfigParser
import openai
import os

pd.set_option('display.max_colwidth',-1)

/tmp/ipykernel_57/2164397915.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth',-1)


## OpenAI account

In [4]:
config=ConfigParser()
config.read('conf/conf.ini')
os.environ["OPENAI_API_KEY"] = config['openai']['apikey']

## Setup weaviate client

In [ ]:
# !rm -rf ~/.local/share/weaviate

In [5]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions(),
    # additional_headers={
    #     "X-OpenAI-Api-BaseURL": os.environ['OPENAI_API_BASE'],
    #     "X-OpenAI-Api-Key": openai.api_key,  # Replace this with your actual key
    # }
)
print(f"Client created? {client.is_ready()}")

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.22.3/weaviate-v1.22.3-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 122
Client created? True


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2023-12-13T17:30:14Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2023-12-13T17:30:14Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2023-12-13T17:30:14Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2023-12-13T17:30:14Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2023-12-13T17:30:14Z"}


## Insert data weaviate

In [6]:
df=pd.read_excel('data/riigikogu/ems_subset.xlsx')
embs=np.load('data/riigikogu/emsb.npy', allow_pickle=True)

/usr/lib/python3/dist-packages/openpyxl/compat/numbers.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  numpy.float,


In [7]:
df.shape, embs.shape

((1000, 1), (1000,))

In [8]:
data=[{'text':row['text_splitted'],'foo':i, 'vector':embs[i], } for i, row in df.iterrows()]

In [9]:
nodes = []
node_prev=None

for idx, sample in enumerate(data):
    node = TextNode(
        #id_=idx,
        text=sample['text'],
    )
    node.embedding=sample['vector']
    # if idx< len(data)-1:
    # node.relationships[NodeRelationship.NEXT]=idx+1
    if idx>0:
        node.relationships[NodeRelationship.PREVIOUS]=RelatedNodeInfo(node_id=node_prev.node_id)
        node_prev.relationships[NodeRelationship.NEXT]=RelatedNodeInfo(node_id=node.node_id)
    nodes.append(node)
    node_prev=node.copy()

In [10]:
node.relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='f113a4ee-b172-48dc-8295-940fa77f5ce7', node_type=None, metadata={}, hash=None)}

In [11]:
NodeRelationship.PREVIOUS

<NodeRelationship.PREVIOUS: '2'>

In [12]:
nodes[2].relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='fa1afa1d-961c-49ed-b8c8-eab6eb71c171', node_type=None, metadata={}, hash=None),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='c676ec60-4143-40b3-bf64-d9736c658cf9', node_type=None, metadata={}, hash=None)}

In [13]:
nodes[3].relationships

{<NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='f1745a15-f365-4f43-a5d7-466fec13fb76', node_type=None, metadata={}, hash=None),
 <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='dd5a66ec-076c-4c7c-847b-67f6267edf56', node_type=None, metadata={}, hash=None)}

In [14]:
RelatedNodeInfo(node_id=node_prev.node_id)

RelatedNodeInfo(node_id='304c4412-f462-4475-8085-995172e4f8fe', node_type=None, metadata={}, hash=None)

## Create vector store

In [15]:
llm = OpenAILLama(model="gpt-3.5-turbo", temperature=0.1)
embed_model=OpenAIEmbedding()

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
#if already run from the start make sure to delete exsiting database
# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Riigikogu", text_key="text")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context, service_context=service_context)

{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"riigikogu_r6cVkHBj1mak","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2023-12-13T17:30:22Z","took":69480}


## If needed setup syntheziser, retriever, post-processors

In [45]:
# Set up the synthesizer
# my_synthesizer = get_response_synthesizer(response_mode="tree_summarize")
# to get only response nodes
my_synthesizer = get_response_synthesizer(response_mode='no_text')

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=20,
)

postprocessor = PrevNextNodePostprocessor(
    docstore=index.docstore,
    num_nodes=1,  # number of nodes to fetch when looking forawrds or backwards
    mode="both",  # can be either 'next', 'previous', or 'both'
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=my_synthesizer,
    node_postprocessors=[
        #postprocessor,
        SimilarityPostprocessor(similarity_cutoff=0.8),
    ]
)

#### example query

In [46]:
query_str = "milliseid ettepanekuid arutatakse, et riigieelarve tasakaalu saada?"
response=query_engine.query(query_str)

In [47]:
response.response

''

In [48]:
response.source_nodes[0].text

'maksud ei ole ainuke võimalus asju mõjutada ja seejärel tulevad otsused rakendamised ja kindlasti järelhindamised mis võib-olla ei ole tänase diskussiooni teema aga ma proovin alustada algusest mis on üldse eesmärk kas eesmärk on saada rohkem raha riigieelarvesse kas probleemiks on väike maksukoormus mina arvan et meie eesmärk on saavutada võimalikult suur heaolu ja seda pidevalt suurendada ühiskonna heaolu saab tugineda üksnes majanduskasvule ja majanduskasv tuleneb ikka ainult ettevõtlusest nii et see on esimene asi mille peale tuleb mõelda järgmine'

In [49]:
len(response.source_nodes)

20

In [24]:
def nodes2df(nodes:list):
    """format list of nodes to pandas dataframe"""
    nodes_fmt=[(n.text, n.id_, n.score) for n in nodes]
    return pd.DataFrame(nodes_fmt, columns=['text','id', 'score'])

In [25]:
# nodes2df(response.source_nodes)

## Setup function calls

In [26]:
import ast
from pydantic import create_model
import inspect, json
from inspect import Parameter

In [27]:
client_oai = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
client_oai

In [28]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [29]:
funcs_ok = {'ask_query_count', 'python'}

#TODOD make queryengine as an argument
def ask_query_count(query_str:str):
    """This function is useful to find count of texts in the database that match query_str. This function gives access to databased.
    Query is made into vector database, returns reponse nodes and counts them
    Query_str should contain words or phrases to search for. It doesn't have to contain AND, OR etc. Could be freeform text"""
    print(f'query string for ask_query_count is {query_str}')
    response=query_engine.query(query_str)
    df_resp=nodes2df(response.source_nodes)
    return str(len(response.source_nodes)), df_resp

In [30]:
def askgpt_func_call(user, system="you are helpful assistant and answer questions about the database that you can access by calling function ask_query_count", model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    c=client_oai.chat.completions.create(model=model, messages=msgs, **kwargs)
    
    if c.choices[0].message.function_call is None:
        return c.choices[0].message.content
    func_call_out, df_resp=call_func(c)
    msgs.append({"role": "function", "name": "python", "content": func_call_out})
    c2=client_oai.chat.completions.create(model=model, messages=msgs)
    return c2.choices[0].message.content, func_call_out, df_resp

In [31]:
resp, func_call_out, df_resp=askgpt_func_call('How many texts mention "sõda" or "kaitsevägi"?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is sõda OR kaitsevägi


'There are a total of 20 texts that mention either "sõda" or "kaitsevägi" in the database.'

In [32]:
df[df.text_splitted.str.contains('sõda')]

,text_splitted
471,lõpuks on 34 inimest nüüd teenistusest vabastatud. näiteks major paas kõikides sõdades osalenud igati laitmatu teenistusega mees lihtsalt lasti lahti kui paneme need numbrid kõik kokku siis tegelikult kaotus personali seas tuleb juba üsna suur kaitseväel on olnud probleeme et kõiki ajateenijaid õpetada sest kaadrit pole piisavalt olnud kuidas kaitsevägi nendes tingimustes pärast seda kolme lainet suudab oma ülesandeid korralikult täita
969,et need muudatused peaks sisse viima veel enne esimest lugemist ehk veel enne tänast päeva loomulikult kuna need muudatused saab sisse viia enne esimest lugemist ainult eelnõu algataja siis andis justiitsministeerium lahkelt selleks ka oma nõusoleku mis tõendab veel kord et mingeid erimeelsusi ega mingit sõda ei õiguskomisjoni istungitel ega töögrupis ei olnud konstruktiivse dialoogiga saime kõik küsimused lahendatud tõepoolest õiguskomisjon arutaski muudetud allikakaitse seaduse eelnõu uuesti juba 5. aprilli istungil see tähendab et allikakaitse seaduse eelnõu muutus ja toodi


In [33]:
df_resp

,text,id,score
0,nime all ta jätkab sõjamuuseumi juures ja minule teadaolevalt on lepingud sõlmitud juba 34 orkestrandiga see orkester nagu me nägime juba kenasti töötas presidendi ametisse astumise ajal nii et ka see on lahendatud te küsite kas ma olen kaitseväe juhatajaga vestelnud nendel teemadel. me oleme kaitseväe juhatajaga vestelnud väga palju eesti kaitseväe teemadel kaitsevõime teemadel ka eri aspektidest mis kaitsevõimekust üldiselt puudutavad olgu see inimesed olgu see võimed kui sellised olgu see kas või koostöö teiste balti riikidega sest tõepoolest,048b3317-a71a-493e-befc-7f8903d73579,0.857824
1,vaktsineerimata siis see on probleem eesti sõjalisele tegevusele kaitseväe tegevusele nii sõja ajal kui ka nende ülesannete täitmisele mis kaitseväel rahuajal on,6d1d1a7a-02c0-45f5-b878-53ee909ccd9e,0.847334
2,"austatud istungi juhataja lugupeetud proua peaminister kevadel kui oli küsimus võimalikus kaitsekulude kärpimises siis kaitsevägi asus seda kiiresti teostama ja sellega seoses koondati päris palju inimesi nüüd loomulikult meie õnneks on selgunud et kaitsekulud praegu tõusevad nad tõusevad 2,31%‑ni skt‑st ja summaarselt on jutt 249,6 miljonist mille eest ma tahan valitsust siiralt tunnustada aga inimesed koondati nüüd edasi seoses vaktsineerimisega terve hulk kaitseväelasi lahkus teenistusest kuna nad tajusid seda kui survet ma toon konkreetse näite kolonelleitnant moora kes seda tegi",ba2e5136-198d-4d63-a6a8-54efeebcf88b,0.844165
3,aitäh härra eesistuja lugupeetud kaja kallas meil on aegrida kujunenud ja see aegrida kaitsejõudude puhul on järgmine teatavasti soovitati 9. mail meie kaitseväelastel riietuda tsiviilriietesse et mitte ärritada 9. maid tähistavaid venekeelseid eesti vabariigi elanikke seejärel tuli uudis et kaotatakse kaplaniteenistus ja pannakse kinni sõjaväeorkester see viga on küll osaliselt parandatud aga mitte täielikult siis tuli uudis et presidendi kantselei auvahtkond hakkab teenistust teostama laigulistes mitte paraadmundrites nüüd on meil tants kaitsejõududes seoses sundvaktsineerimisega mis on väga selgelt põhjustanud elukutselistes,6e073a0a-e68c-4520-9846-249c5f09f39d,0.843919
4,lugupeetud peaministrile on küsimus. lugupeetud juhataja hea peaminister juttu oli siin paari sõnaga kaitseväe orkestrist siin on tegelikult üks probleem. nimelt kaitseväe orkestrist oleme me juba varem rääkinud ja oleme kuulnud et asi hakkab nagu korda minema orkester läheb muuseumi alla ja asi on tore aga ma ise suhtlen hästi tihedalt nende pillimeestega kes mängisid nii kaitseväe orkestris kui ka endises piirivalveorkestris no piirivalveorkestriga on selge temast on tehtud nüüd poliitiline orkester tallinna linnavalitsuse alla ja see ei ole enam,9b788559-dadc-4afe-9d6d-062c3a575d92,0.843653
5,algab meie kaitse ju ka meie naabrite kaitsmisest olukord on olnud ju ärev seetõttu oleme valitsuses iga nädal rääkinud julgeolekuteemadel olukord mis on valgevene-leedu piiril valgevene-läti piiril või valgevene-poola piiril mõjutab otseselt ka meie julgeolekut siin. nii et mina kaitseväe juhatajat tema tegevuses igapidi toetan ta on teinud julgeid samme ja seda kõike selleks et just eesti kaitsevõimet tugevdada ka kaitseväelaste vaktsineerimisele eelnenud riskianalüüs näitas et kui kaitseväelased nii suurtes hulkades jäävad haigeks või peavad jääma isolatsiooni kuna nad on,957bf475-fee8-4d23-a077-88677d2f7b4f,0.838523
6,sõjaväelastes pahameelt rahulolematust vabatahtlikku lahkumist ja sunniviisilist lahkumist ma tuletan siinkohal meelde ka seda et kaitsejõudude praegune juht kindral herem oli operatiivosakonnas teeninud riigireetur deniss metsavase ajal kaitseväe peastaabi ülem seesama herem on kinnitanud kaitseväe peastaabi ülemaks ja kaitseväe juhataja asetäitjaks kindral palmi kes on saanud negatiivse atestatsiooni ja on minu andmetel isegi istunud kainestusmajas ma küsin teilt kas kõik need asjaolud kõik need operatsioonid orkestrite mundrite ja muuga on olnud teile eelnevalt teada juhataja helistab kella. kas te olete,

In [119]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis mainitakse riiki Ukraina?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is riik Ukraina


'Ukraina mainitakse kahes tekstis.'

In [120]:
df[df.text_splitted.str.contains('ukraina')]

,text_splitted
191,arvatud rikastes põhjamaades see pärsib meie konkurentsivõimet ja paneb surve alla palgad kui kõik muud sisendhinnad on naabrite omadest kõrgemad siis ainus koht kust kokku hoida on töötajad just sellepärast nõuavad mõned tööandjad meile odavat tööjõudu ukrainast elektri ja gaasi hinda langetades anname ettevõtjatele võimaluse tõsta palkasid kaotamata konkurentsis välisturgudel eesmärk peaks olema muuta eesti kõige odavama energiaga riigiks meie piirkonnas milliseid maksupoliitilisi muudatusi me veel peame vajalikuks tee- ja veokimaks ei suurenda oluliselt riigieelarvet küll aga vähendab meie transpordiettevõtete
267,kolm maksuastet 16% 24% ja 33% lühikest aega ka 50% kui tuli võimule mart laar siis ta kehtestas ühtse 27%-lise maksu mis tähendas seda et rikkamad hakkasid vähem maksma ja väiksema sissetulekuga inimesed tunduvalt rohkem ma ei saa siiamaani aru miks me ei taha minna sama teed nagu muu euroopa miks me tahame seista kogu aeg ühel pulgal venemaaga kus on nagu meil proportsionaalne ja ühetaoline tulumaks täpselt samuti on ka bulgaarias rumeenias ukrainas ja ma ei tea bahama saartel


In [121]:
df_resp

,text,id,score
0,arvatud rikastes põhjamaades see pärsib meie konkurentsivõimet ja paneb surve alla palgad kui kõik muud sisendhinnad on naabrite omadest kõrgemad siis ainus koht kust kokku hoida on töötajad just sellepärast nõuavad mõned tööandjad meile odavat tööjõudu ukrainast elektri ja gaasi hinda langetades anname ettevõtjatele võimaluse tõsta palkasid kaotamata konkurentsis välisturgudel eesmärk peaks olema muuta eesti kõige odavama energiaga riigiks meie piirkonnas milliseid maksupoliitilisi muudatusi me veel peame vajalikuks tee- ja veokimaks ei suurenda oluliselt riigieelarvet küll aga vähendab meie transpordiettevõtete,975f70e9-ae31-46a3-8607-b6c33004a52e,0.809876
1,aitäh härra eesistuja lugupeetud kaja kallas meil on aegrida kujunenud ja see aegrida kaitsejõudude puhul on järgmine teatavasti soovitati 9. mail meie kaitseväelastel riietuda tsiviilriietesse et mitte ärritada 9. maid tähistavaid venekeelseid eesti vabariigi elanikke seejärel tuli uudis et kaotatakse kaplaniteenistus ja pannakse kinni sõjaväeorkester see viga on küll osaliselt parandatud aga mitte täielikult siis tuli uudis et presidendi kantselei auvahtkond hakkab teenistust teostama laigulistes mitte paraadmundrites nüüd on meil tants kaitsejõududes seoses sundvaktsineerimisega mis on väga selgelt põhjustanud elukutselistes,42717d0c-c9cd-4d6f-927d-ec60935a38bd,0.802319


In [122]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse kunstist või kunstiteosest või kunstilisest objektist?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is kunst OR kunstiteos OR kunstiline objekt


'Andmebaasi põhjal ei räägita üheski tekstis kunstist, kunstiteosest või kunstilisest objektist.'

In [99]:
df[df.text_splitted.str.contains('kunst')]

,text_splitted
929,aitäh lugupeetud juhataja lugupeetud minister ka minu küsimus puudutab võlaõigusseaduse muudatusi mis on seotud kahju hüvitamisega eesti kohtupraktikas on üks selline juhtum et kadri k filmi m. tõlkes great ehk 'suurepärane' puhul ei olnud kohtul muud võimalust kui see film ära keelata kuigi tegelikult oleks lahenduseks olnud ka kahju hüvitamine kas selle eelnõu puhul oleks võimalik öelda et edaspidi sellistel juhtumitel mingi kultuuri- või ka kunstilise objekti filmi ükskõik mis asja ärakeelamise asemel saaks kasutada kahju hüvitamist kui ei siis


In [124]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse arvutitest või IT-st?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is arvutitest OR IT


'Andmebaasi põhjal on teada, et 16 tekstis räägitakse arvutitest või IT-st.'

In [123]:
df[df.text_splitted.str.contains('arvuti')]

,text_splitted
13,peale 3 miljardit tundi mis tegi ühe maksumaksja kohta keskmiseks ajakuluks 27 tundi aastas eesti on aga maailmas läbi löönud tugeva it-riigina ja siin esitatakse üle 98% tuludeklaratsioonidest interneti teel täiendav maksuaste ei ole arvutiprogrammi jaoks mingisugune takistus eesti on olnud aastaid sissetulekute jaotuse poolest pigem euroopa liidu ebavõrdsemate riikide hulgas seda näitab gini indeks mis sobib kõige paremini iseloomustama ühiskondade sotsiaalset ebavõrdsust ja kihistumist mida madalam on gini indeksi väärtus seda võrdsem on ühiskonna majanduslik ja sotsiaalne seisund arenenud
75,ei ole varsti enam optimaalne lahendus ja riigid peaksid mõtlema muule lahendusele see eristamine on kulukas see on võib-olla ebavajalik ja see ei pruugi mingist hetkest alates ka õiglustundega vastavuses olla peame arvestama ka sellega et diginomaadluski on kasvamas diginomaadid on sellised inimesed kuidas öelda mees ja arvuti tüüpi oü-de asutajad kes reisivad mööda maailma ja kelle töötegemise asukoht ei ole üldse oluline nemad teevad tööd internetis see tähendab seda et riikide sotsiaalkindlustussüsteemid on löögi all neli aastat tagasi hinnati


In [125]:
df_resp

,text,id,score
0,peale 3 miljardit tundi mis tegi ühe maksumaksja kohta keskmiseks ajakuluks 27 tundi aastas eesti on aga maailmas läbi löönud tugeva it-riigina ja siin esitatakse üle 98% tuludeklaratsioonidest interneti teel täiendav maksuaste ei ole arvutiprogrammi jaoks mingisugune takistus eesti on olnud aastaid sissetulekute jaotuse poolest pigem euroopa liidu ebavõrdsemate riikide hulgas seda näitab gini indeks mis sobib kõige paremini iseloomustama ühiskondade sotsiaalset ebavõrdsust ja kihistumist mida madalam on gini indeksi väärtus seda võrdsem on ühiskonna majanduslik ja sotsiaalne seisund arenenud,a6f870d6-93c2-420e-bcf2-ccb6063de212,0.806443
1,täpsustav küsimus andres metsoja palun,0372d4dd-0910-4d21-8539-99d3a06a3cd8,0.805742
2,küll aga on võimalik muuta riigiteenuseid läbipaistvamaks ja automaatsemaks kui me mõistlikult arendaksime oma e-riiki ja kasutaksime rohkem tehisintellekti aitäh,ee362567-ff18-4339-8fc3-c38fbf2bdacd,0.805299
3,austatud minister võib-olla remargi korras mina selle kongressi osas nii optimistlik ei ole aga selle kohta ma küsida ei taha ma küsin hoopis kas see seadusmuudatus hõlbustab spordiklubide statistiliste ja muude andmete kogumist praegu peab maksu- ja tolliametile oma aruanded esitama ühel kuupäeval statistikaametile hoopis teisel kuupäeval ka tallinna linnavalitsus koostab oma spordiregistrit ja üleeilseks pidi selle ära tegema enamik spordiklubisid on suhteliselt väiksed ja raamatupidajad töötavad lepingu alusel neile on suhteliselt suur koormus täita iga kuu mingisuguseid pabereid kas,767c25c2-f453-4a34-b29e-80dca3241601,0.804524
4,"aitäh ma kahjuks ei ole andmekaitsespetsialist ma olen maksuspetsialist aga ma olen suhteliselt kindel ütleme 99,9% kindel et te olete siiski andnud oma nõusoleku nende andmete kasutamiseks võib-olla see on lihtsalt kuskil kirjas väga väikses šriftis ega ei ole ka üheselt arusaadav aga teie küsimus on väga huvitav illustreerimaks olukorda kuidas lähenetakse digihiidude ja andmete kui uue nafta käsitusele märgin et mõned riigid eelistavad öelda et see et inimesed saavad tasuta kasutada facebooki google'it jne tähendab seda et toimub partnertehing",be1c1187-da73-4485-b000-9e984a115672,0.803139
5,representatiivsus samuti andmete kvaliteet kuna puudub konkreetne seaduslik alus andmete kogumiseks ja töötlemiseks ei suuda spordiregister täies mahus veel oma eesmärke täita infoks niipalju et on olemas leping statistikaameti ning spordikoolituse ja -teabe sihtasutuse vahel alla kirjutatud 19. jaanuaril 2010. aastal mis annab edaspidi kõik õigused üle statistikaametile praegu ei kogu ükski riigi ega kohaliku omavalitsuse andmekogu andmeid spordiorganisatsioonide -ehitiste ja treenerite kohta spordikoolide kohta kogub andmeid eesti hariduse infosüsteem ehk ehis spordialasid käsitlevaid andmeid sealt aga ei leia on,e8a31825-0bef-4a7f-bfd7-bbf1936c2715,0.802895
6,on praeguseks juba osaliselt valminud veel arendamist vajava osa tarbeks rahuldas spordikoolituse ja -teabe sihtasutus riigi infosüsteemide keskusele majanduskeskkonna arendamise rakenduskava infoühiskonna edendamise prioriteetse suuna avatud taotlusvooru eeltaotluse mahus 1 704 725 krooni millest omafinantseering moodustab 15% ehk 255 709 krooni aitäh,8de22720-d311-4825-b4d0-7b6b1e775c67,0.802875
7,austatud istungi juhataja head kolleegid annan keskerakonna fraktsiooni nimel sisse järjekordse arupärimise sotsiaalminister hanno pevkurile mis puudutab vaesuse temaatikat toimetulekupiiri küsime miks ei ole kaks viimast aastat enam arvutatud toimetulekupiiri suurust küsime miks ei ole meile teada ostukorvi suurus kas seda ei tahetagi näidata me teame et kahel viimasel aastal eriti aastal 2008 oli väga suur inflatsioon ja on selge et senise maksumusega ostukorvi enam needsamad kaubad ja teenused ei mahu ilmselt sellepärast seda ei tahetagi meile teada anda ja,b4ef0481-6d1d-4b38-a982-767c69b549cc,0.802722
8,siiski facebooki ja google'i ja ka kõikide teiste bilansis ja nad saavad nende andmete kasutamise või ne

In [ ]:
resp, func_call_out, df_resp=askgpt_func_call('Kui mitmes tekstis räägitakse infospsteemide arendamisest?',
                functions=[schema(ask_query_count)])
resp

query string for ask_query_count is infospsteemide arendamisest


'Andmebaasis esineb infospsteemide arendamisest rääkivaid tekste 7 korda.'

In [127]:
df_resp

,text,id,score
0,täpsustav küsimus andres metsoja palun,0372d4dd-0910-4d21-8539-99d3a06a3cd8,0.820274
1,head ametikaaslased nüüd on suurepärane võimalus kõigil meil üle anda eelnõusid ja arupärimisi palun riigikogu kõnetooli kõiki kes soovivad üle anda eelnõusid ja arupärimisi palun riigikogu kõnetooli helir-valdor seederi,9fea70d9-7f57-4fe2-9555-23280dfeda24,0.804349
2,on praeguseks juba osaliselt valminud veel arendamist vajava osa tarbeks rahuldas spordikoolituse ja -teabe sihtasutus riigi infosüsteemide keskusele majanduskeskkonna arendamise rakenduskava infoühiskonna edendamise prioriteetse suuna avatud taotlusvooru eeltaotluse mahus 1 704 725 krooni millest omafinantseering moodustab 15% ehk 255 709 krooni aitäh,8de22720-d311-4825-b4d0-7b6b1e775c67,0.803356
3,eks see viita ka omamoodi sellele inforuumile mis meile on siin loodud,261f6190-95dc-4218-8ebb-caf964abcefa,0.801747
4,spordiregistri loomine muudab sporditeabe kogumise süsteemsemaks ja efektiivsemaks see teave esitatakse ühtselt ühte andmekogusse mis vähendab teabe andjate koormust spordivaldkonda puudutava teabe koondamine võimaldab efektiivsemalt läbi viia riiklikke statistilisi vaatlusi samuti langetada olulisi spordipoliitilisi otsuseid teavet saavad kasutada ministeeriumide eelkõige kultuuriministeeriumi sotsiaalministeeriumi haridus- ja teadusministeeriumi ning kaitseministeeriumi ametnikud omavalitsuste spetsialistid ja volikogude liikmed statistikaameti spetsialistid eesti olümpiakomitee spordiorganisatsioonid ja kohalike omavalitsuste ametnikud saavad seega finantseerimisotsuste langetamisel ja objektiivse info hankimisel kasutada spordiregistri andmekogu väikese remargina ütlen et paar aastat tagasi,48a8bc17-6435-4652-b8f4-b69e2e8610e8,0.800792
5,aitäh lugupeetud riigikogu esimees austatud peaminister tõepoolest minu küsimus puudutab pagulastele elamistingimuste loomist õigem oleks öelda et elamisvõimaluste otsimist liigub info et vabariigi valitsus on andnud korralduse või oleks äkki õigem öelda et otsib võimalusi afgaanidele korterite leidmiseks on olnud kuulda et on soov leida korter kuni 3000 afgaanile sealhulgas kuni 2500‑le virumaal mingile osale jõgevamaal ja kuskile veel eesti on väga väike ja kõik tunnevad kõiki selline info liigub inimeste hulgas päris laias ringis ma olen seda kuulnud vähemalt,6e05d67a-04f6-4fa3-8c32-e1af6281261b,0.800417
6,nendel teemadel härra heremiga vestelnud ja juhtinud tema tähelepanu sellele et tema tegevus on süstemaatiline kaitseväe moraali ja kaitsetahte õõnestamine,7fb28bf8-3327-4c6e-bc62-697a3b1f742d,0.800148
